In [4]:
import random
import time
import redis
import json
#!pip install mysql-connector-python
import mysql.connector
r = redis.Redis(host='192.168.80.166', port=6379)

In [5]:
#function from other team member
def road(type3,level,city):
    db = mysql.connector.connect(user='root', password='1234', host='10.120.28.5',port='3306', database="final",charset = 'utf8')
    cursor = db.cursor()    
    
    type3_dict = {"mountaineering":" > 150 ","running":" < 150 "}
    x = type3_dict.get(type3)
    
    level_dict ={"strong":" '高' ","medium":" '中' ","weak":" '低' "}
    y = level_dict.get(level)
    
    city_dict = {"TPE":"%台北%", "NTPC":"%新北%", "KEL":"%基隆%", "TYN":"%桃園%", "HSZ":"%新竹%",
                "ZMI":"%苗栗%", "ILA":"%宜蘭%", "TXG":"%台中%", "CHW":"%彰化%", "NTC":"%南投%", 
                 "YUN":"%雲林%", "CYI":"%嘉義%", "TNN":"%台南%", "KHH":"%高雄%", "PIF":"%屏東%",
                "PEH":"%澎湖%"}
    z = city_dict.get(city)
    
    cursor.execute("SELECT final.mountain_basic.url, final.mountain_basic.標題, final.mountain_basic.評分, final.mountain_basic.里程, final.mountain_basic.海拔高度, final.mountain_basic.難易度, final.mountain_city.所在縣市 FROM final.mountain_basic LEFT JOIN final.mountain_city ON final.mountain_basic.標題 = final.mountain_city.標題 WHERE final.mountain_basic.高度落差"+ x +"AND final.mountain_city.所在縣市 LIKE '"+ z +"' AND final.mountain_basic.難易度 LIKE" + y)  
    
    res_list = []
    for x in cursor:
        #url
        a = x[0]
        #標題
        b = x[1]
        #評分
        c = x[2]
        #里程
        d = x[3]
        #海拔高度
        e = x[4]
        #難易
        f = x[5]
        #縣市
        g = x[6]
        res = {'url': a, 'name': b, 'score':c, 'mileage':d,'high':e, 'level':f, 'city':g}
        res_list.append(res)
    db.close()
    return res_list

In [ ]:
#saving random one route of different type, strength ,city
city_dict = {"TPE":"台北", "NTPC":"新北", "KEL":"基隆", "TYN":"桃園", "HSZ":"新竹",
                "ZMI":"苗栗", "ILA":"宜蘭", "TXG":"台中", "CHW":"彰化", "NTC":"南投", 
                 "YUN":"雲林", "CYI":"嘉義", "TNN":"台南", "KHH":"高雄", "PIF":"屏東",
                "PEH":"澎湖"}
while True:
    for city in ['TPE', 'NTPC', 'KEL', 'TYN', 'HSZ', 'ILA', 'ZMI', 'TXG', 'CHW', 'NTC', 'YUN', 'CYI', 'TNN', 'KHH', 'PIF', 'PEH']:
        for strength in ["strong", "medium", "weak"]:   
            for exercise in ["running", "mountaineering"]:
                route = road(exercise, strength, city)
                #if result of route doesn't exist
                if route == []:
                    r.hmset(exercise+"_"+strength+"_"+city, {city: city_dict[city], strength: strength, 'info': 'Not find route'})
                #result for route is a list, random.choice one of list to set in redis
                elif route != []:
                    r.hmset(exercise+"_"+strength+"_"+city, {city: city_dict[city], strength: strength, 'info': str(random.choice(route))})
                print(exercise+"_"+strength+"_"+city)
                print(exercise, ":", r.hmget(exercise+"_"+strength+"_"+city, [city, strength, 'info'])[2].decode())
                print('--------------------------------------------------TYPE--------------------------------------------------')
            print('--------------------------------------------------STRENGTH--------------------------------------------------')
        print('--------------------------------------------------CITY--------------------------------------------------')
    time.sleep(5)
    print('--------------------------------------------------UPDATE--------------------------------------------------')

running_strong_TPE
running : Not find route
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_strong_TPE
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1218', 'name': '陽明山東西大縱走(超健腳加強版)', 'score': 5, 'mileage': '39.2公里', 'high': '5~1120公尺', 'level': '高', 'city': '台北市中山區'}
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_medium_TPE
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1059', 'name': '北投大砲岩、石壇山步道', 'score': 4, 'mileage': '單程0.3公里', 'high': '190~243公尺', 'level': '中', 'city': '台北市北投區'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_medium_TPE
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&ac

running_strong_HSZ
running : Not find route
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_strong_HSZ
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=540', 'name': '武陵四秀登山步道', 'score': 4, 'mileage': '29.2公里', 'high': '1980~3524公尺', 'level': '高', 'city': '新竹縣尖石鄉'}
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_medium_HSZ
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=953', 'name': '泰崗溫泉步道', 'score': 4, 'mileage': '單程1.6公里', 'high': '834~922公尺', 'level': '中', 'city': '新竹縣尖石鄉'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_medium_HSZ
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=

running_strong_CHW
running : Not find route
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_strong_CHW
mountaineering : Not find route
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_medium_CHW
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1252', 'name': '八卦山一線天步道', 'score': 4, 'mileage': '2.6公里', 'high': '69~125公尺', 'level': '中', 'city': '彰化縣彰化市'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_medium_CHW
mountaineering : Not find route
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH-------------------------------------------------

mountaineering_strong_TNN
mountaineering : Not find route
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_medium_TNN
running : Not find route
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_medium_TNN
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=784', 'name': '獵鷹尖一線天步道', 'score': 4, 'mileage': '3.9公里', 'high': '847~1000公尺', 'level': '中', 'city': '台南市楠西區'}
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_weak_TNN
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1318', 'name': '漁光島步道', 'score': 4, 'mileage': '2.3公里',

running_medium_TPE
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1059', 'name': '北投大砲岩、石壇山步道', 'score': 4, 'mileage': '單程0.3公里', 'high': '190~243公尺', 'level': '中', 'city': '台北市北投區'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_medium_TPE
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1356', 'name': '帕米爾公園、碧山、白石湖山、大邱田天然石壁洞', 'score': 4, 'mileage': '4.1公里', 'high': '312~517公尺', 'level': '中', 'city': '台北市士林區'}
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_weak_TPE
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=982', 'name': '芝山岩步道群', 'score': 4, 'mileage': '3.5公里', 'high': '0~46公尺', 'level': '低', 'city': '台北市士林區'}
-----------------------------------------

...

running_medium_TNN
running : Not find route
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_medium_TNN
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=780', 'name': '三腳南山登山步道', 'score': 3, 'mileage': '單程2.9公里', 'high': '450~1087公尺', 'level': '中', 'city': '台南市南化區'}
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_weak_TNN
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1272', 'name': '五間登山步道', 'score': 4, 'mileage': '單程1.6公里', 'high': '152~292公尺', 'level': '低', 'city': '台南市龍崎區'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_weak_TNN
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=816

mountaineering_medium_TPE
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1016', 'name': '七星連走(七星瀑布/七星公園/七星東峰/七星主峰/七星錐/七星南峰/七星池)', 'score': 4, 'mileage': '6.8公里', 'high': '750~1120公尺', 'level': '中', 'city': '台北市北投區'}
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_weak_TPE
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1185', 'name': '土地公嶺古道', 'score': 4, 'mileage': '單程0.9公里', 'high': '91~133公尺', 'level': '低', 'city': '台北市信義區'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_weak_TPE
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1170', 'name': '瑪礁古道', 'score': 4, 'mileage': '5.2公里', 'high': '422~780公尺', 'level': '低', 'city': '台北市士林區'}
----------------

mountaineering_medium_HSZ
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=932', 'name': '李崠山、馬望僧侶山O型', 'score': 3, 'mileage': '15.4公里', 'high': '1308~1914公尺', 'level': '中', 'city': '新竹縣尖石鄉'}
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_weak_HSZ
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1593', 'name': '巨埔步道、龍頸步道', 'score': 0, 'mileage': '2.4公里', 'high': '143~267公尺', 'level': '低', 'city': '新竹縣新埔鎮'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_weak_HSZ
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1171', 'name': '東穗山登山步道', 'score': 4, 'mileage': '單程1.2公里', 'high': '1370~1638公尺', 'level': '低', 'city': '新竹縣尖石鄉'}
---------------------------------

running_medium_CHW
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1252', 'name': '八卦山一線天步道', 'score': 4, 'mileage': '2.6公里', 'high': '69~125公尺', 'level': '中', 'city': '彰化縣彰化市'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_medium_CHW
mountaineering : Not find route
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_weak_CHW
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1202', 'name': '彰化溪州公園步道', 'score': 4, 'mileage': '5.9公里', 'high': '35~39公尺', 'level': '低', 'city': '彰化縣溪州鄉'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_weak_CHW
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=432', 'n

mountaineering_medium_TNN
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=308', 'name': '關仔嶺大凍山步道', 'score': 4, 'mileage': '單程3公里', 'high': '601~1241公尺', 'level': '中', 'city': '台南市白河區'}
--------------------------------------------------TYPE--------------------------------------------------
--------------------------------------------------STRENGTH--------------------------------------------------
running_weak_TNN
running : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1262', 'name': '菁埔埤濕地步道', 'score': 4, 'mileage': '1.5公里', 'high': '36~49公尺', 'level': '低', 'city': '台南市六甲區'}
--------------------------------------------------TYPE--------------------------------------------------
mountaineering_weak_TNN
mountaineering : {'url': 'https://hiking.biji.co/index.php?q=trail&act=detail&id=1073', 'name': '鈺鼎步道', 'score': 4, 'mileage': '單程1.6公里', 'high': '170~430公尺', 'level': '低', 'city': '台南市南化區'}
-----------------------------------------------

In [33]:
# #saving random one route of different type, strength ,city
# city_dict = {"TPE":"台北", "NTPC":"新北", "KEL":"基隆", "TYN":"桃園", "HSZ":"新竹",
#                 "ZMI":"苗栗", "ILA":"宜蘭", "TXG":"台中", "CHW":"彰化", "NTC":"南投", 
#                  "YUN":"雲林", "CYI":"嘉義", "TNN":"台南", "KHH":"高雄", "PIF":"屏東",
#                 "PEH":"澎湖"}
# for city in ['TPE', 'NTPC', 'KEL', 'TYN', 'HSZ', 'ILA', 'ZMI', 'TXG', 'CHW', 'NTC', 'YUN', 'CYI', 'TNN', 'KHH', 'PIF', 'PEH']:
#     for strength in ["strong", "medium", "weak"]:   
#         print(city, '|', strength, '|', len(road("hill", strength, city)))
#         print('----------------------------------------------------------------------------------------------')

TPE | strong | 4
----------------------------------------------------------------------------------------------
TPE | medium | 58
----------------------------------------------------------------------------------------------
TPE | weak | 39
----------------------------------------------------------------------------------------------
NTPC | strong | 4
----------------------------------------------------------------------------------------------
NTPC | medium | 197
----------------------------------------------------------------------------------------------
NTPC | weak | 106
----------------------------------------------------------------------------------------------
KEL | strong | 0
----------------------------------------------------------------------------------------------
KEL | medium | 14
----------------------------------------------------------------------------------------------
KEL | weak | 5
-----------------------------------------------------------------------------------